In [4]:
import numpy as np 
import torch
import torchvision

In [6]:
BATCH_SIZE = 64
NUM_CLASSES = 10
LEARNING_RATE = 0.001
NUM_EPOCHS = 20

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
all_transforms = torchvision.transforms.Compose([
                                                    torchvision.transforms.Resize((32, 32)),
                                                    torchvision.transforms.ToTensor(),
                                                    torchvision.transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                                                     std=[0.2023, 0.1994, 0.2010])
                                                ])

train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True,
                                             transform=all_transforms,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False,
                                            transform=all_transforms,
                                            download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

170499072it [06:28, 438846.29it/s]                                                                                     


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [25]:
class ConvNeuralNet(torch.nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv_layer3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = torch.nn.Linear(1600, 128)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(128, num_classes)
        
    def forward(self, X):
        X = torch.tensor(X)
        out = self.conv_layer1(X)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
        
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        
        return out

In [28]:
model = ConvNeuralNet(NUM_CLASSES).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, weight_decay=0.005, momentum=0.9)
total_step = len(train_loader)

In [29]:
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epochs [{epoch+1}/{NUM_EPOCHS}], Loss : {loss.item():.4f}")

C:\Users\LENOVO\anaconda3\envs\tfpt\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epochs [1/20], Loss : 1.6865
Epochs [2/20], Loss : 1.3029
Epochs [3/20], Loss : 0.9874
Epochs [4/20], Loss : 0.6892
Epochs [5/20], Loss : 1.4208
Epochs [6/20], Loss : 1.1669
Epochs [7/20], Loss : 0.8028
Epochs [8/20], Loss : 1.6310
Epochs [9/20], Loss : 0.8647
Epochs [10/20], Loss : 0.8814
Epochs [11/20], Loss : 0.5013
Epochs [12/20], Loss : 0.8425
Epochs [13/20], Loss : 0.9546
Epochs [14/20], Loss : 0.5414
Epochs [15/20], Loss : 0.8212
Epochs [16/20], Loss : 0.9632
Epochs [17/20], Loss : 0.6999
Epochs [18/20], Loss : 1.1074
Epochs [19/20], Loss : 0.6369
Epochs [20/20], Loss : 0.3139


In [30]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))




C:\Users\LENOVO\anaconda3\envs\tfpt\lib\site-packages\ipykernel_launcher.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Accuracy of the network on the 50000 train images: 81.808 %
